<a href="https://colab.research.google.com/github/HuaInZu/easy-stable-diffusion-goormide/blob/main/easy_stable_diffusion_goormide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <font color="red">실행하는 방법</font>

- [원본 원클릭Colab 아카라이브 게시글](https://arca.live/b/aiart/60472214)
- [원클릭 구름IDE 아카라이브 게시글](https://arca.live/b/aiart/66555000)
- [백업용 git 레포지토리](https://github.com/HuaInZu/easy-stable-diffusion-goormide)
<br>본 원클릭 구름IDE는 원클릭Colab의 코드를 구름IDE환경에 맞게 수정한 버전입니다.
<br>컨테이너 신규 생성 기준으로 작성되었으므로 일부 라이브러리 버전이 다른 환경에서 사용시 오류가 날 수 있습니다.

<br><font color="red" size="4">**하단의 첫 실행시 터미널 실행은 일부 라이브러리 버전 문제로 필수적으로 수행해야합니다.**</font>

1. 첫 실행시 <font color="red">**하단의 터미널에**</font>
<br>sudo add-apt-repository ppa:deadsnakes/ppa
<br>sudo apt install -y python3.10
<br>sudo apt-get install -y python3.10-distutils
<br>curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10
<br>python3 -m pip install --upgrade pip
<br>python3 -m pip install wget
<br>apt install -y libgl1-mesa-glx
<br>python3.10 -m pip install matplotlib
<br>python3.10 -m pip install IPython
<br><font color="red">**넣어 각 요소 버전 맞추기 필수, 미수행시 버전문제로 동작불가**</font>, 상단 첫 줄은 중간에 엔터 한 번 필요
<br>(23/06/18 baf6946e06249c5af9851c60171692c44ef633e0 커밋기준)
<br>
1. 하단의 실행 칸을 클릭 후 상단에 있는 <font color="red">**▶▶ 버튼**</font> 클릭(▶Run 버튼으로 사용이 가능하나 실행된걸 재실행시 기본 동작폴더인 SD폴더안에 경로가 중첩되어 동작할 수 있음)
    <font color="orange" size="2">오류가 발생하면 빨간색 경고 메세지가 나옴 ([예시 이미지](https://cdn.discordapp.com/attachments/872959812407816235/1029468903216267294/2022-10-12_04-01-54_02b5_librewolf.png))</font>
    <font color="orange" size="2">정상적으로 완료되면 초록색 메세지가 나옴 ([예시 이미지](https://media.discordapp.net/attachments/872959812407816235/1029468507328499784/2022-10-12_04-00-19_02b4_librewolf.png))</font>
    <font color="red" size="3">처음 실행할 때 약 20분 정도 소요됨</font>
1. 원클릭 Colab의 기본설정은 import 파트 다음에서 진행
1. WebUI가 켜진 상태에서 컨테이너를 종료해도 WebUI는 동작하나 에러가 날 수 있으므로 권장하지 않음

## import(필요 라이브러리 호출)

In [ ]:
import os
import sys
import shutil
import tempfile
import re
import shlex
import json
import requests
import torch
import torchvision
import wget

from typing import Dict, Union, Callable, Tuple, List, Set
from subprocess import Popen, PIPE, STDOUT
from distutils.spawn import find_executable
from importlib.util import find_spec
from pathlib import Path
from io import FileIO
from datetime import datetime

OPTIONS = {}

## 모델 종류 지정
실행시 불러올 모델명 작성시 해당 모델을 불러옴, 비워두면 실행하지 않음(마지막 로드된 모델 로드) / 기본 **''**, 작성양식은 하단 참조
<br>Anything-v4.0-pruned-fp32.safetensors

In [ ]:
LOAD_MODEL = ''
OPTIONS['LOAD_MODEL'] = LOAD_MODEL

## Xformers 사용여부 확인
장점: 이미지 생성 속도 개선 가능성 있음 / 단점: 출력한 그림의 질이 조금 떨어질 수 있음
<br>사용시 True, 미사용시 False / **기본 True**

In [ ]:
USE_XFORMERS = True
OPTIONS['USE_XFORMERS'] = USE_XFORMERS

## Gradio 터널을 사용여부 확인
장점: 따로 설정할 필요가 없어 편리함 / 단점: 접속이 느리고 끊키거나 버튼이 안 눌리는 등 오류 빈도가 높음
<br>사용시 True, 미사용시 False / **기본 True**

In [ ]:
USE_GRADIO = True
OPTIONS['USE_GRADIO'] = USE_GRADIO

## Gradio 인증 정보
Gradio 접속 시 사용할 사용자 아이디와 비밀번호 입력
<br>GRADIO_USERNAME 에 `user1:pass1,user,pass2`처럼 입력하면 여러 사용자 추가 가능
<br>GRADIO_USERNAME 을 ''상태로 두면 인증 과정을 사용하지 않음 / **기본 'gradio'**
<br>GRADIO_PASSWORD 을 ''상태로 두면 자동으로 비밀번호를 생성함 / **기본 ''**

In [ ]:
GRADIO_USERNAME = 'gradio'
GRADIO_PASSWORD = ''
GRADIO_PASSWORD_GENERATED = False
OPTIONS['GRADIO_USERNAME'] = GRADIO_USERNAME
OPTIONS['GRADIO_PASSWORD'] = GRADIO_PASSWORD

## ngrok API 키
ngrok 터널에 사용할 API 토큰
<br>https://arca.live/b/aiart/60683088 참고
<br>장점: 접속이 빠른 편이고 타임아웃이 거의 발생하지 않음 / 단점: 계정을 만들고 API 토큰을 직접 입력해줘야함
<br>NGROK_API_TOKEN 을 ''상태로 두면 ngrok 터널을 비활성화함 / **기본 ''**

In [ ]:
NGROK_API_TOKEN = ''
NGROK_URL = None
OPTIONS['NGROK_API_TOKEN'] = NGROK_API_TOKEN

## WebUI 업데이트 여부
SD WebUI의 GIT파일을 가져올지에 대한 여부, 업데이트를 하지 않으면 환경을 고정해서 쓸 수 있으므로 문제가 생길 가능성이 줄어듬
<br>**기본 True**, 항상 모든 데이터를 업데이트 / False, 업데이트 수행하지 않음

In [ ]:
OPTIONS['GIT_UPDATE'] = True

## WebUI 레포지토리 커밋 해시
SD WebUI의 최신 버전에서 오류가 발생할 때 과거 커밋 해시 값(영문과 숫자로된 난수 값)을 붙여넣은 뒤 실행하면 과거 버전을 사용할 수 있음
<br>[레포지토리 커밋 목록] - https://github.com/AUTOMATIC1111/stable-diffusion-webui/commits/master
<br>입력 란을 비워두면 가장 최신 커밋을 가져옴 / **기본''**

In [ ]:
REPO_COMMIT = ''
OPTIONS['REPO_COMMIT'] = REPO_COMMIT

## WebUI 인자
[사용할 수 있는 인자 목록](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Command-Line-Arguments-and-Settings)
<br>직접 인자를 넣어 동작하고 싶을 때 사용
<br><font color="red">**주의**</font>: 작성된 것이 있다면 기본 실행 인자를 넣지 않으므로 SD WebUI에대해 잘 모르면 건드리지 말것

In [ ]:
ARGS = ''
OPTIONS['ARGS'] = shlex.split(ARGS)

## WebUI 추가 인자
추가적인 인자를 넣고 싶을때 사용, 종류는 상단의 목록 확인 / **기본'--skip-torch-cuda-test'**
<br>주요 사용 인자
<br>--no-half-vae = vae 관련 오류시 사용
<br>--lowrem = 대용량 이미지 제작시 사용

In [ ]:
EXTRA_ARGS = '--skip-torch-cuda-test'
OPTIONS['EXTRA_ARGS'] = shlex.split(EXTRA_ARGS)

## WebUI 레포지토리 주소
<font color="red">**주소 문제가 생길 경우 외에는 건드리지 말것**</font>

In [ ]:
REPO_URL = 'https://github.com/AUTOMATIC1111/stable-diffusion-webui.git'
OPTIONS['REPO_URL'] = REPO_URL

## 기본 수행 폴더 지정
<font color="red">**가급적 수정 금지**</font> 임베딩, 모델, 결과와 설정 파일 등이 영구적으로 보관될 디렉터리 경로

In [ ]:
WORKSPACE = 'SD'
OPTIONS['WORKSPACE'] = WORKSPACE

## 실행

In [ ]:
# 로그 파일
LOG_FILE: FileIO = None

# 로그 HTML 위젯
LOG_WIDGET = None

# 로그 HTML 위젯 스타일
LOG_WIDGET_STYLES = {
    'wrapper': {
        'overflow-x': 'auto',
        'max-width': '100%',
        'padding': '1em',
        'background-color': 'black',
        'white-space': 'pre',
        'font-family': 'monospace',
        'font-size': '1em',
        'line-height': '1.1em',
        'color': 'white'
    },
    'dialog': {
        'display': 'block',
        'margin-top': '.5em',
        'padding': '.5em',
        'font-weight': 'bold',
        'font-size': '1.5em',
        'line-height': '1em',
        'color': 'black'
    }
}
LOG_WIDGET_STYLES['dialog_success'] = {
    **LOG_WIDGET_STYLES['dialog'],
    'border': '3px dashed darkgreen',
    'background-color': 'green',
}
LOG_WIDGET_STYLES['dialog_warning'] = {
    **LOG_WIDGET_STYLES['dialog'],
    'border': '3px dashed darkyellow',
    'background-color': 'yellow',
}
LOG_WIDGET_STYLES['dialog_error'] = {
    **LOG_WIDGET_STYLES['dialog'],
    'border': '3px dashed darkred',
    'background-color': 'red',
}

PYTHON_BINARY = 'python'

def setup_environment():
    global LOG_WIDGET

    # 노트북 환경이라면 로그 표시를 위한 HTML 요소 만들기
    if 'ipykernel' in sys.modules:
        try:
            from IPython.display import display
            from ipywidgets import widgets

            LOG_WIDGET = widgets.HTML()
            LOG_WIDGET.blocks = []
            display(LOG_WIDGET)

        except ImportError:
            pass

    if not OPTIONS['USE_GRADIO'] and not OPTIONS['NGROK_API_TOKEN']:
        log('선택된 터널링 서비스가 없습니다.', styles={'color': 'red'})

    # 자동으로 인자 넣을 시 확장기능 상시 사용하도록
    if len(OPTIONS['ARGS']) < 1:
        OPTIONS['EXTRA_ARGS'] += ['--enable-insecure-extension-access']

        if not torch.cuda.is_available():
            log(
                'GPU 런타임이 아닙니다. CPU로 시도하나 GPU 컨테이너로 다시 실행하는 것을 권장합니다.',
                styles={'color': 'red'})
            OPTIONS['EXTRA_ARGS'] += [
                '--skip-torch-cuda-test',
                '--no-half',
                '--opt-sub-quad-attention'
            ]

    chdir(OPTIONS['WORKSPACE'])

    global PYTHON_BINARY

    if find_executable(PYTHON_BINARY):
        PYTHON_BINARY = 'python3.10'


# ==============================
# 로그
# ==============================


def format_styles(styles: dict) -> str:
    return ';'.join(map(lambda kv: ':'.join(kv), styles.items()))


def format_list(value):
    if isinstance(value, dict):
        return '\n'.join(map(lambda kv: f'{kv[0]}: {kv[1]}', value.items()))
    else:
        return '\n'.join(value)


def render_log() -> None:
    html = f'''<div style="{format_styles(LOG_WIDGET_STYLES['wrapper'])}">'''

    for block in LOG_WIDGET.blocks:
        styles = {
            'display': 'inline-block',
            **block['styles']
        }
        child_styles = {
            'display': 'inline-block',
            **block['child_styles']
        }

        html += f'<span style="{format_styles(styles)}">{block["msg"]}</span>\n'

        if block['max_childs'] is not None and len(block['childs']) > 0:
            html += f'<div style="{format_styles(child_styles)}">'
            html += ''.join(block['childs'][-block['max_childs']:])
            html += '</div>'

    html += '</div>'

    LOG_WIDGET.value = html


def log(
    msg: str,
    styles={},
    newline=True,

    parent=False,
    parent_index: int = None,
    child_styles={
        'padding-left': '1em',
        'color': 'gray'
    },
    max_childs=0,

    print_to_file=True,
    print_to_widget=True
) -> Tuple[None, int]:

    # 기록할 내용이 ngrok API 키와 일치한다면 숨기기
    # TODO: 더 나은 문자열 검사, 원치 않은 내용이 가려질 수도 있음
    if OPTIONS['NGROK_API_TOKEN'] != '':
        msg = msg.replace(OPTIONS['NGROK_API_TOKEN'], '**REDACTED**')

    if newline:
        msg += '\n'

    # 파일에 기록하기
    if print_to_file and LOG_FILE:
        if parent_index and msg.endswith('\n'):
            LOG_FILE.write('\t')
        LOG_FILE.write(msg)
        LOG_FILE.flush()

    # 로그 위젯에 기록하기
    if print_to_widget and LOG_WIDGET:
        # 부모 로그가 없다면 새 블록 만들기
        if parent or parent_index is None:
            LOG_WIDGET.blocks.append({
                'msg': msg,
                'styles': styles,
                'childs': [],
                'child_styles': child_styles,
                'max_childs': max_childs
            })
            render_log()
            return len(LOG_WIDGET.blocks) - 1

        # 부모 로그가 존재한다면 추가하기
        LOG_WIDGET.blocks[parent_index]['childs'].append(msg)
        render_log()
        return

    print('\t' if parent_index else '' + msg, end='')
    return


def log_trace() -> None:
    import sys
    import traceback

    # 스택 가져오기
    ex_type, ex_value, ex_traceback = sys.exc_info()

    styles = {}

    # 오류가 존재한다면 메세지 빨간색으로 출력하기
    # https://docs.python.org/3/library/sys.html#sys.exc_info
    # TODO: 오류 유무 이렇게 확인하면 안될거 같은데 일단 귀찮아서 대충 써둠
    if ex_type is not None:
        styles = LOG_WIDGET_STYLES['dialog_error']

    parent_index = log('보고서를 만들고 있습니다...', styles)

    # 오류가 존재한다면 오류 정보와 스택 트레이스 출력하기
    if ex_type is not None:
        log(parent_index=parent_index, msg=f'{ex_type.__name__}: {ex_value}')
        log(
            parent_index=parent_index,
            msg=format_list(
                map(
                    lambda v: f'{v[0]}#{v[1]}\n\t{v[2]}\n\t{v[3]}',
                    traceback.extract_tb(ex_traceback))
            )
        )

    # 로그 파일이 없으면 보고하지 않기
    # TODO: 로그 파일이 존재하지 않을 수가 있나...?
    if not LOG_FILE:
        log('로그 파일이 존재하지 않습니다, 보고서를 만들지 않습니다')
        return

    # 로그 위젯이 존재한다면 보고서 올리고 내용 업데이트하기
    if LOG_WIDGET:
        # 이전 로그 전부 긁어오기
        logs = ''
        with open(LOG_FILE.name) as file:
            logs = file.read()

        # 로그 업로드
        # TODO: 업로드 실패 시 오류 처리
        res = requests.post('https://hastebin.com/documents',
                            data=logs.encode('utf-8'))
        url = f"https://hastebin.com/raw/{json.loads(res.text)['key']}"

        # 기존 오류 메세지 업데이트
        LOG_WIDGET.blocks[parent_index]['msg'] = '\n'.join([
            '오류가 발생했습니다, 아래 주소를 복사해 보고해주세요',
            f'<a target="_blank" href="{url}">{url}</a>',
        ])

        render_log()


# ==============================
# 서브 프로세스
# ==============================
running_subprocess = None


def execute(
    args: Union[str, List[str]],
    parser: Callable = None,
    summary: str = None,
    hide_summary=False,
    print_to_file=True,
    print_to_widget=True,
    throw=True,
    **kwargs
) -> Tuple[str, Popen]:

    global running_subprocess

    # 이미 서브 프로세스가 존재한다면 예외 처리하기
    if running_subprocess and running_subprocess.poll() is None:
        raise Exception('이미 다른 하위 프로세스가 실행되고 있습니다')

    if isinstance(args, str) and 'shell' not in kwargs:
        kwargs['shell'] = True

    # 서브 프로세스 만들기
    running_subprocess = Popen(
        args,
        stdout=PIPE,
        stderr=STDOUT,
        encoding='utf-8',
        **kwargs)
    running_subprocess.output = ''

    # 로그에 시작한 프로세스 정보 출력하기
    formatted_args = args if isinstance(args, str) else ' '.join(args)
    summary = formatted_args if summary is None else f'{summary}\n   {formatted_args}'

    running_subprocess.parent_index = log(
        f'=> {summary}',
        styles={'color': 'yellow'},
        max_childs=5)

    # 프로세스 출력 위젯에 리다이렉션하기
    while running_subprocess.poll() is None:
        # 출력이 비어있다면 넘어가기
        line = running_subprocess.stdout.readline()
        if not line:
            continue

        # 프로세스 출력 버퍼에 추가하기
        running_subprocess.output += line

        # 파서 함수 실행하기
        if callable(parser):
            try:
                if parser(line):
                    continue
            except:
                log_trace()

        # 프로세스 출력 로그하기
        log(
            line,
            newline=False,
            parent_index=running_subprocess.parent_index,
            print_to_file=print_to_file,
            print_to_widget=print_to_widget)

    # 변수 정리하기
    output = running_subprocess.output
    returncode = running_subprocess.poll()

    # 로그 블록 업데이트
    if LOG_WIDGET:
        if returncode == 0:
            # 성공적으로 프로세스가 종료됐을 때
            if hide_summary:
                # 현재 로그 블록 숨기기 (제거하기)
                del LOG_WIDGET.blocks[running_subprocess.parent_index]
            else:
                # 현재 로그 텍스트 초록색으로 변경하고 프로세스 출력 숨기기
                LOG_WIDGET.blocks[running_subprocess.parent_index]['styles']['color'] = 'green'
                LOG_WIDGET.blocks[running_subprocess.parent_index]['max_childs'] = None
        else:
            # 현재 로그 텍스트 빨간색으로 변경하고 프로세스 출력 모두 표시하기
            LOG_WIDGET.blocks[running_subprocess.parent_index]['styles']['color'] = 'red'
            LOG_WIDGET.blocks[running_subprocess.parent_index]['max_childs'] = 0

        # 로그 블록 렌더링
        render_log()

    # 오류 코드를 반환했다면
    if returncode != 0 and throw:
        raise Exception(f'프로세스가 {returncode} 코드를 반환했습니다')

    return output, returncode

# ==============================
# 작업 경로
# ==============================


def chdir(cwd: os.PathLike) -> None:
    global LOG_FILE

    cwd = Path(cwd).absolute()

    # 작업 경로 변경
    old_cwd = Path.cwd().absolute()
    cwd.mkdir(0o777, True, True)
    os.chdir(cwd)

    # 기존 로그 파일 옮기기
    os.makedirs('logs', exist_ok=True)

    log_path = Path('logs', datetime.strftime(
        datetime.now(), '%Y-%m-%d_%H-%M-%S.log'))

    if LOG_FILE:
        LOG_FILE.close()
        Path(old_cwd, LOG_FILE.name).rename(log_path)

    LOG_FILE = log_path.open('a')

    # 현재 환경 출력
    import platform
    log(platform.platform())
    log(f'Python {platform.python_version()}')
    log(f'torch {torch.__version__}')
    log(f'torchvision {torchvision.__version__}')
    log(str(Path.cwd()))


def has_python_package(pkg: str, executable: str = None) -> bool:
    if not executable:
        return find_spec(pkg) is not None

    _, rc = execute(
        [
            executable, '-c',
            f'''
            import importlib
            import sys
            sys.exit(0 if importlib.find_loader({shlex.quote(pkg)}) else 0)
            '''
        ],
        throw=False)

    return True if rc == 0 else False


# ==============================
# 파일 다운로드
# ==============================
def download(url: str, target: str, ignore_aria2=False, **kwargs):
    # 파일을 받을 디렉터리 만들기
    Path(target).parent.mkdir(0o777, True, True)

    execute(
        [
            'wget',
            '-O', target,
            url
        ],
        hide_summary=True)

def has_checkpoint() -> bool:
    for p in Path('repository', 'models', 'Stable-diffusion').glob('**/*'):
        if p.suffix != '.ckpt' and p.suffix != '.safetensors':
            continue
        return True
    return False


# ==============================
# WebUI 레포지토리 및 종속 패키지 설치
# ==============================
def patch_webui_repository() -> None:

    # 기본 파일 만들기
    for path, content in {
        'repository/config.json': json.dumps({
            'CLIP_stop_at_last_layers': 2
        }),
        'repository/ui-config.json': json.dumps({
            'txt2img/Prompt/value': 'best quality, masterpiece, high resolution',
            'txt2img/Negative prompt/value': '(worst quality, low quality:1.3)',
            'txt2img/Sampling steps/value': 28,
            'txt2img/Width/value': 512,
            'txt2img/Height/value': 768,
            'txt2img/CFG Scale/value': 12
        }),
        'repository/styles.csv': ''
    }.items():
        path = Path(path)
        if not path.exists():
            path.write_text(content)

    # Gradio 에서 앱이 위치한 경로와 다른 장치에 있는 내부 파일 접근시 발생하던 ValueError 를 해결하는 스크립트
    download(
        'https://raw.githubusercontent.com/toriato/easy-stable-diffusion/main/scripts/patches.py',
        'repository/scripts/fix_gradio_route.py',
        )

def setup_webui() -> None:
    need_clone = True

    path = Path('repository')

    # 이미 디렉터리가 존재한다면 정상적인 레포인지 확인하기
    if path.is_dir() and OPTIONS['GIT_UPDATE'] == True:
        try:
            # 사용자 파일만 남겨두고 레포지토리 초기화하기
            # 기존 git reset --hard head && git pull 적용시 간혹 모든 개인파일을 날려버리는 상황이 생겨 pull만 하도록
            execute(
                'git pull',
                summary='레포지토리를 업데이트 합니다',
                cwd='repository')

            need_clone = False

        except:
            log('레포지토리가 잘못됐습니다, 디렉터리를 제거합니다')

    if need_clone and OPTIONS['GIT_UPDATE'] == True:
        # 실제 레포지토리 경로 가져오기
        # 코랩 환경에선 레포지토리 경로를 심볼릭 링크하기 때문에 경로를 가져와야함
        if path.is_symlink():
            path = os.readlink(path)

        shutil.rmtree(path, ignore_errors=True)
        execute(
            ['git', 'clone', OPTIONS['REPO_URL'], str(path)],
            summary='레포지토리를 가져옵니다')

    # 특정 커밋이 지정됐다면 체크아웃하기
    if OPTIONS['REPO_COMMIT'] != '':
        execute(
            ['git', 'checkout', OPTIONS['REPO_COMMIT']],
            summary=f"레포지토리를 {OPTIONS['REPO_COMMIT']} 커밋으로 되돌립니다",
            cwd=path)

    patch_webui_repository()

    # 체크포인트 모델이 존재하지 않는다면 기본 모델 받아오기
    if not has_checkpoint():
        log('체크포인트 모델이 존재하지 않습니다. 모델을 받아옵니다.')
        for file in [
            {
                'url': 'https://huggingface.co/Kaeya/aichan_blend/resolve/main/AbyssOrangeMix2_hard-pruned.safetensors',
                'target': 'repository/models/Stable-diffusion/AbyssOrangeMix2_hard-pruned.safetensors',
            },
            {
                'url': 'https://huggingface.co/Kaeya/aichan_blend/resolve/main/vae/BerrysMix.vae.safetensors',
                'target': 'repository/models/VAE/BerrysMix.vae.safetensors',
            }
        ]:
            download(**file)
        log('체크포인트 모델 다운 완료')


def parse_webui_output(line: str) -> bool:
    global NGROK_URL

    # 하위 파이썬 실행 중 오류가 발생하면 전체 기록 표시하기
    # TODO: 더 나은 오류 핸들링, 잘못된 내용으로 트리거 될 수 있음
    if LOG_WIDGET and 'Traceback (most recent call last):' in line:
        LOG_WIDGET.blocks[running_subprocess.parent_index]['max_childs'] = 0
        render_log()
        return

    # 내부에서 재시작할 때 ngrok 세션이 존재해도 다시 열려고 시도하는데
    # 사용 중인 토큰이 무료 플랜이면 한 세션만 열 수 있다며 뻑 나는 경우가 있음
    # 애초에 세션이 여러 개 생기면 안되긴 하지만... 일단 자동좌가 수정해주거나 PR 넣는 방법 밖엔 없을듯
    # NGROK_URL 변수가 비어있을 때만 오류 처리하도록 수정함
    if NGROK_URL == None and line == 'Invalid ngrok authtoken, ngrok connection aborted.\n':
        raise Exception('ngrok 인증 토큰이 잘못됐습니다, 올바른 토큰을 입력하거나 토큰 값 없이 실행해주세요')

    # 로컬 웹 서버가 열렸을 때
    if line.startswith('Running on local URL:'):
        if GRADIO_PASSWORD_GENERATED:
            # gradio 인증
            log(
                '\n'.join([
                    'Gradio 인증 비밀번호가 자동으로 생성됐습니다',
                    f"아이디: {OPTIONS['GRADIO_USERNAME']}",
                    f"비밀번호: {OPTIONS['GRADIO_PASSWORD']}"
                ]),
                LOG_WIDGET_STYLES['dialog_success'],
                print_to_file=False)

        # ngork
        if OPTIONS['NGROK_API_TOKEN'] != '':
            # 이전 로그에서 ngrok 주소가 표시되지 않았다면 ngrok 관련 오류 발생한 것으로 판단
            if NGROK_URL == None:
                raise Exception('ngrok 터널을 여는 중 알 수 없는 오류가 발생했습니다')

            if LOG_WIDGET:
                log(
                    '\n'.join([
                        '성공적으로 ngrok 터널이 열렸습니다',
                        NGROK_URL if LOG_WIDGET is None else f'<a target="_blank" href="{NGROK_URL}">{NGROK_URL}</a>',
                    ]),
                    LOG_WIDGET_STYLES['dialog_success'])
            else:
                log(f'성공적으로 ngrok 터널이 열렸습니다: {NGROK_URL}')

        return

    # 외부 주소 출력되면 성공적으로 실행한 것으로 판단
    if line.startswith('ngrok connected to') or line.startswith('Running on public URL:'):
        url = re.search(r'https?://.+', line)[0]

        # gradio 는 웹 서버가 켜진 이후 바로 나오기 때문에 사용자에게 바로 보여줘도 상관 없음
        if 'gradio' in url:
            if LOG_WIDGET:
                log(
                    '\n'.join([
                        '성공적으로 Gradio 터널이 열렸습니다',
                        '<a target="_blank" href="https://arca.live/b/aiart/60683088">Gradio 는 느리고 버그가 있으므로 ngrok 사용을 추천합니다</a>',
                        f'<a target="_blank" href="{url}">{url}</a>',
                    ]),
                    LOG_WIDGET_STYLES['dialog_warning'])
            else:
                log(f'성공적으로 Gradio 터널이 열렸습니다: {url}')

        # ngork 는 우선 터널이 시작되고 이후에 웹 서버가 켜지기 때문에
        # 미리 주소를 저장해두고 이후에 로컬호스트 주소가 나온 뒤에 사용자에게 알려야함
        if 'ngrok-free.app' in url:
            NGROK_URL = url

        return


def start_webui(args: List[str] = OPTIONS['ARGS'], env: Dict[str, str] = None) -> None:
    global GRADIO_PASSWORD_GENERATED

    # 로드할 모델 변경
    if OPTIONS['LOAD_MODEL'] != '':
        model_path = 'repository/config.json'
        with open(model_path, 'r') as file:
            data = json.load(file)
        data['sd_model_checkpoint'] = OPTIONS['LOAD_MODEL']
        with open(model_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, indent="\t")
        log(f'불러올 모델을 {LOAD_MODEL}로 변경했습니다.')
        log('')

    # 기본 환경 변수 만들기
    if env is None:
        env = {
            **os.environ,
            'PYTHONUNBUFFERED': '1',
            'HF_HOME': Path('cache', 'huggingface').absolute()
        }

    # 기본 인자 만들기
    if len(args) < 1:
        # xformers
        if OPTIONS['USE_XFORMERS'] and torch.cuda.is_available():
            args += ['--xformers']

        # gradio
        if OPTIONS['USE_GRADIO']:
            args += ['--share']

        # gradio 인증
        if OPTIONS['GRADIO_USERNAME'] != '' and OPTIONS['USE_GRADIO']:
            # 다계정이 아니고 비밀번호가 없다면 무작위로 만들기
            if OPTIONS['GRADIO_PASSWORD'] == '' and ';' not in OPTIONS['GRADIO_USERNAME']:
                from secrets import token_urlsafe
                OPTIONS['GRADIO_PASSWORD'] = token_urlsafe(8)
                GRADIO_PASSWORD_GENERATED = True

            args += [
                f'--gradio-auth',
                OPTIONS['GRADIO_USERNAME'] +
                ('' if OPTIONS['GRADIO_PASSWORD'] ==
                 '' else ':' + OPTIONS['GRADIO_PASSWORD'])
            ]

        # ngrok
        if OPTIONS['NGROK_API_TOKEN'] != '':
            args += [
                '--ngrok', OPTIONS['NGROK_API_TOKEN'],
                '--ngrok-region', 'jp'
            ]

    # 추가 인자
    args += OPTIONS['EXTRA_ARGS']

    execute(
        [PYTHON_BINARY, 'launch.py', *args],
        parser=parse_webui_output,
        cwd='repository',
        env=env)


try:
    setup_environment()
    setup_webui()
    start_webui()

# ^c 종료 무시하기
except KeyboardInterrupt:
    pass

except:
    # 로그 위젯이 없다면 평범하게 오류 처리하기
    if not LOG_WIDGET:
        raise

    log_trace()
